# Resume NER
## Extract Information from Resumes using NER (Named Entity Recognition)

### Part 2 - NER with Spacy
In this second part of the challenge, we will be using the preprocessed data from part one to start training NER models. We will be using spacy (https://spacy.io/) here to "get our feet wet" with NER, as training spacy can be reasonably done on our laptops and does not yet necessarily require a GPU. Spacy is a powerful, effective, and resource-efficient NLP library - It might surprise us with its performance on the challenge!

We will run spacy's pretrained models on our data to get a feel for NER, and then we will perform some additional preprocessing on our data before we start training our own NER model using the labelled entities we have identified in part one. 
We will also explore evaluation metrics for NER, and decide how we want to quantify the performance of our trained models. 

* *If you need help setting up python or running this notebook, please get help from the  assistants to the professor*
* *It might be helpful to try your code out first in a python ide like pycharm before copying it an running it here in this notebook*
* *For solving the programming tasks, use the python reference linked here (Help->Python Reference) as well as Web-searches.* 

##### Reload preprocessed data
Here, we will load the data we saved in part one and save it to a variable. Provide code below to load the data and store it as a list in a variable. (Hint - use 'open' and the json module)

In [1]:
## import json module
import json
path = "data/converted_resumes.json"
## TODO open file load as json and store in "resumes" variable
with open(path) as f:
    resumes = json.load(f)
## TODO print length of loaded resumes list to be sure everything ok
print(len(resumes))


690


##### Take Spacy for a spin
Before we train our own NER model to recognize the resume-specific entities we want to capture, let's see how spacy's pretrained NER models do on our data. These pretrained models can't recognize our entities yet, but let's see how they do. Run the next code block to load spacy's English language model 


In [2]:
import spacy
nlp = spacy.load('en')
print(nlp)


Now we get the EntityRecognizer in the loaded nlp pipeline and display the labels it supports

In [3]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('LANGUAGE', 'ORG', 'WORK_OF_ART', 'LAW', 'PERSON', 'DATE', 'CARDINAL', 'FAC', 'EVENT', 'ORDINAL', 'PERCENT', 'NORP', 'QUANTITY', 'LOC', 'PRODUCT', 'MONEY', 'GPE', 'TIME')


##### Question: What do the 'GPE', 'FAC' and 'NORP' labels stand for? (Tipp: use either the spacy.explain method, or google the spacy.io api docs) 
https://spacy.io/api/annotation#named-entities

* GPE: Countries, cities, states.
* FAC: Buildings, airports, highways, bridges, etc.
* NORP: Nationalities or religious or political groups.


In [4]:
### TODO  if you choose to use spacy's 'explain' method to get the answer to the question above, provide your code here
## print description of entities using spacy explain
print("GPE: ", spacy.explain("GPE"))
print("FAC: ", spacy.explain("FAC"))
print("NORP: ", spacy.explain("NORP"))


GPE:  Countries, cities, states
FAC:  Buildings, airports, highways, bridges, etc.
NORP:  Nationalities or religious or political groups


As we can see, the entities are different than the entities we will train our custom model on. 
##### Question: what entities do you think this model will find in an example resume?
PERSON, ORG, GPE/LOC

Now we will work with one of our resumes, and get spacy to tell us what entities it recognizes. Complete the code block below to get a single resume text out of our resume list. 

In [5]:
### TODO get a single resume text and print it out
restxt = resumes[42][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))


Navas Koya
Test Engineer
Mangalore, Karnataka - Email me on Indeed: indeed.com/r/Navas-Koya/23c1e4e94779b465
Willing to relocate to: Mangalore, Karnataka - Bangalore, Karnataka - Chennai, Tamil Nadu
WORK EXPERIENCE
System Engineer
Infosys -
August 2014 to Present
.NET application Maintenance and do the code changes if required
Test Engineer
Infosys -
June 2015 to February 2016
PrProject 2:
Title: RBS W&G Proving testing.
Technology: Manual testing
Role: Software Test Engineer
Domain: Banking
Description:
Write test cases & descriptions. Review the entries. Upload and map the documents into
HP QC. Execute the testing operations in TPROD mainframe. Upload the result in QC along with
the proof.
Roles and Responsibilities:
•Prepared the Test Scenarios
•Prepared and Executed Test Cases
•Performed functional, Regression testing, Sanity testing.
•Reviewed the Test Reports and Preparing Test Summary Report.
•Upload Test cases to the QC.
•Execute in TPROD Mainframe.
•Defect Track and Report.
Te

Extracting entities with spacy is easy with a pretrained model. We simply call the model (here 'nlp') with our text to get a spacy Document. See https://spacy.io/api/doc for more detail. 

Execute the code below to process the resume txt.

In [6]:
doc = nlp(restxt)


The doc object has a list of entities predicted by spacy 'ents'. We would like to loop through all of these entities and print their label and associated text to see what spacy predicted for this resume.

Complete the code below to do this. You will probably need to google the spacy api docs to find the solution (Tipp: look for 'Doc.ents'). Also, trying code in your ide (for example pycharm) before copying it here might help with exploring and debugging to find the solution. 

In [7]:
##TODO loop through the doc's entities, and print the label and text for each entity found. 
for ent in doc.ents:
    print(ent, " : ", ent.label_)


Navas Koya  :  PERSON
Test Engineer  :  PERSON
Karnataka - Email  :  PERSON
Karnataka - Bangalore  :  PERSON
Karnataka - Chennai  :  ORG
August 2014  :  DATE
Maintenance  :  PERSON
Test Engineer  :  PERSON
June 2015 to February 2016  :  DATE
RBS W&G Proving  :  ORG
HP QC  :  ORG
TPROD  :  ORG
Responsibilities  :  PERSON
Executed Test Cases  :  ORG
the Test Reports  :  ORG
Preparing Test Summary Report  :  ORG
Track and Report  :  ORG
Infosys Limited -

  :  PERSON
August 2014  :  DATE
May 2015  :  DATE
1  :  CARDINAL
CAWP  :  PERSON
Role  :  PERSON
Software Test Executive  :  ORG
Admin  :  ORG
annual  :  DATE
Business Requirement  :  ORG
Functional  :  ORG
Responsibilities  :  PERSON
the Test Scenarios
  :  ORG
Executed Test Cases  :  ORG
the Test Reports  :  ORG
Preparing Test Summary Report  :  ORG
Track and Report  :  ORG
Computer Applications

Mangalore University  :  ORG
June 2011  :  DATE
April 2014  :  DATE
SQL Server, Css  :  ORG
74%  :  PERCENT
Milagres College  :  ORG
Kallian

##### Questions: What is your first impression of spacy's NER based on the results above? Does it seem accurate/powerfull?
The spacy's NER is not quite accurate.

##### Does it make many mistakes? Do some entity types seem more accurate than others?
Yes, there are some errors. The DATE entity is very accurate, the ORG, PRODUCT and PERSON are not.



Now as a comparison, we will list the entities contained in the resume's original annotated training data (remember, the existing annotations were created by a human-annotator, and not predicted by a machine like the entities predicted above) 

Complete the code below to do the following: 
* Access the 'entities' list of the example resume you chose, loop through the entities and print them out. 
* *Tip: one entity in the list is a tuple with the following structure: (12,1222,"label") where the first element is the start index of the entity in the resume text, the second element is the end index, and the third element is the label.
* Use this Tip to print out a formatted list of entities 



In [8]:
##TODO print original entities for one resume
res = resumes[42]
restext = res[0]
labeled_ents = res[1]['entities']
## TDOD print out formatted list of entity labels and text
for ent in labeled_ents:
    print("{} {}\n".format(ent[2], restext[ent[0]:ent[1]]))

Skills SKILL SET • ASP.NET, C# • QA tools

• Coding and modularization • Excellent communication skills

• VB, VB.net, ASP • Technical specifications creation

• HTML • System backups

• Sql server 2005, Oracle • System upgrades

• Java/C/C++ • Excellent problem-solving abilities

Navas Najeer Koya 3

Location Mangalore

Skills C# (Less than 1 year), .NET, SQL Server, Css, Html5


Graduation Year  2014

Location Mangalore

Location Mangalore

Degree Bachelor of Computer Application

Graduation Year  2014

Companies worked at Infosys

Designation Test Engineer


Companies worked at Infosys

Designation Test Engineer


Graduation Year  2014

Companies worked at Infosys

Designation System Engineer

Location Mangalore

Location Mangalore

Designation Test Engineer


Name Navas Koya



As we already know, the annotated entities in the training data are different than the entities spacy can recognize with it's pretrainied models, so we need to train a custom NER model. We will get started with that now. 

##### Prepare Training Data for NER model training
We need to do some more preprocessing of our training data before we can train our model.

Remember the entity labels you chose in part 1 of the challenge? We will be training a model to predict those entities.
As a first step, we will gather all resumes that contain at least one training annotation for those entities.

Complete and execute the code below to gather your training data. 

In [9]:
##TODO Store the entity labels you want to train for as array in chosen_entity_labels
chosen_entity_labels = ["Skills", "Name", "Location"]

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
## TODO use the gather candidates methods and store result in training_data variable
training_data = gather_candidates(resumes, chosen_entity_labels)
print("Gathered {} training examples".format(len(training_data)))


Gathered 503 training examples


Now we have those training examples which contain the entities we are interested in. Do you have at least a few hundred examples? If not, you might need to re-think the entities you chose or try just one or two of them and re-run the notebooks. It is important that we have several hundred examples for training (e.g. more than 200. 3-500 is better). 

##### Remove other entity annotations from training data
Now that we have our training data, we want to remove all but relevant (chosen) entity annotations from this data, so that the model we train will only train for our entities. Complete and execute the code below to do this. 

In [10]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable 
X = [[res[0], dict(entities=filter_ents(res[1]['entities'], chosen_entity_labels))] for res in training_data]
X[42]


["Nazish Alam\nConsultant - SAP ABAP\n\nGhaziabad, Uttar Pradesh - Email me on Indeed: indeed.com/r/Nazish-Alam/\nb06dbac9d6236221\n\nWilling to relocate to: Delhi, Delhi - Noida, Uttar Pradesh - Lucknow, Uttar Pradesh\n\nWORK EXPERIENCE\n\nConsultant\n\nSAP ABAP -  Noida, Uttar Pradesh -\n\nNovember 2016 to Present\n\nCredence Systems, Noida\n\nCredence Systems is IT Infrastructure Management Company, offers end-to-end solutions.\nCombining deep domain expertise with new technologies and a cost effective on-site/ offshore\nmodel. Helping companies integrate key business processes, improving their operational\nefficiencies and extracting, better business value from their investment.\n\nPROJECT UNDERTAKEN\nClient ECC Version Role and Responsibilities\nWelspun Group Plate & Coil Mills Division\nSAP ECC 6.0\n\nConsultant\n\nSAP ABAP -\n\nJanuary 2016 to Present\n\nReports:\n• Designed technical program specifications based on business requirements.\n• Generated basic lists and Interactive

##### Remove resumes that cause errors in spacy
Depending on what entities you chose, some of the resumes might cause errors in spacy. We don't need to get into details as to why, suffice to say it has to do with whitespace and syntax in the entity annotations. If these resumes are not removed from our training data, spacy will throw an exception during training, so we need to remove them first. 

We will use the remove_bad_data function below to do this. This function does the following:
* calls train_spacy_ner with debug=True and n_iter=1. This causes spacy to process the documents one-by-one, and gather the documents that throw an exception in a list of "bad docs" which it returns. 
* You will complete the function to remove any baddocs (returned by remove_bad_data) from your training data list. 

You may or may not have any bad documents depending on the entities you chose. In any case, there should not be more than a dozen or so bad docs.  

In [11]:
from spacy_train_resume_ner import train_spacy_ner

def remove_bad_data(training_data):
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    ## training data is list of lists with each list containing a text and annotations
    ## baddocs is a set of strings/resume texts.
    ## TODO complete implementation to filter bad docs and store filter result (good docs) in filtered variable
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method. It may take a few minutes for the method to complete.
## you will know it is complete when the print output above. 
X = remove_bad_data(X)


Created blank 'en' model
Exception thrown when processing doc:
("Neeraj Dwivedi\nSenior Sales Executive - Kansai Nerolac Paints Ltd\n\nMumbai, Maharashtra - Email me on Indeed: indeed.com/r/Neeraj-Dwivedi/8f053ed44cdef8b2\n\nWORK EXPERIENCE\n\nSenior Sales Executive\n\nKansai Nerolac Paints Ltd -  Mumbai, Maharashtra -\n\nAugust 2017 to Present\n\nIncreased the sales from -22% to +6% through existing dealers and prospecting new dealers to\nachieve the target within 6 months of joining.\n• Maintaining excellent relations with dealers to increase revenue by 10%.\n• Management of sales team (up to 3 members)\n• Achieved monthly targets amounting to 40 - 60 lakhs monthly with timely collection.\n• Maintain accounts clarity with dealers and ensure that the credit notes reach the dealers on\ntime.\n• Business development by conducting regular meetings with influencers such as architects,\nbuilding contractors, housing societies, painters.\n• Ensuring proper visibility of the products at vari

In [12]:
print(len(X))


498


##### Question: How many bad docs did you have? What is the size of your new (filtered) training data? 
Five bad docs, 498 datasets in a new training data are present.


##### Train/Test Split
Now before we train our model, we have to split our available training data into training and test sets. Splitting our data into train and test (or holdout) datasets is a fundamental technique in machine learning, and essential to avoid the problem of overfitting.
Before we go on, you should get a grasp of how train/test split helps us avoid overfitting. Please take the time now to do a quick web search on the topic. There are many resources available. You should search for "train test validation overfitting" or some subset of those terms.

Here are a few articles to start with:
* https://machinelearningmastery.com/a-simple-intuition-for-overfitting/
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation (Note: you are free to install scikit learn and use the train_test_split method documented here, but it is not necessary. It is the concept that is important)

##### Question: What is overfitting and how does doing a train/test split help us avoid overfitting when training our models? Please answer in your own words. 
An overfitting means the model contains more parameters to fit than the data provides. 


Now that we understand why we do a train/test split, we will write some code that splits our data into train and test sets. Usually we want around 70-80% of the data for train, and the rest for test. 
##### TODO: Complete the code below to create a train and test dataset


In [14]:
##TODO complete the implementation  of the train test split function below
def train_test_split(X,train_percent):
    train_size = int(len(X) * (train_percent/100))
    train = X[:train_size]
    test = X[train_size:]
    return train,test
## TODO chose train size percent and call train test split, storing results in "train" and "test" variables.
train,test = train_test_split(X, 70)
## TODO use python assert to assert that the size of train and test sets add up to the size of all the data 
assert len(X) == len(train) + len(test)
print("Test: ", len(test), ", Train: ", len(train))


Test:  150 , Train:  348


##### Train a spacy ner model with our training data
OK, now it is (finally) time to train our own custom NER model using spacy. Because our training data has been preprocessed to only include annotations for the entities we are interested in, the model will only be able to predict/extract those entities. 
*Depending on your computer, this step may take a while.* Training 20 epochs (iterations) using 480 training examples takes around 10 minutes on my machine (core i7 CPU). You will see output like *Losses {'ner':2342.23342342}* after each epoch/iteration. The default number of iterations is 20, so you will see this output 20 times. When this step is done, we will use the trained ner model to perform predictions on our test data in our test set.  

In [15]:
## run this code to train a ner model using spacy
custom_nlp,_= train_spacy_ner(train,n_iter=20)


Created blank 'en' model
Losses {'ner': 37202.443002108776}
Losses {'ner': 17735.210161813673}
Losses {'ner': 13144.159044783013}
Losses {'ner': 14216.834321991566}
Losses {'ner': 12073.707825156314}
Losses {'ner': 11121.387102987483}
Losses {'ner': 12520.128805040964}
Losses {'ner': 9805.358840240777}
Losses {'ner': 7938.908639285404}
Losses {'ner': 8074.862212184387}
Losses {'ner': 7513.0699158959615}
Losses {'ner': 7866.972667430955}
Losses {'ner': 8478.064681466687}
Losses {'ner': 7341.443052584882}
Losses {'ner': 8162.654087951412}
Losses {'ner': 7293.600492337902}
Losses {'ner': 6607.747966766714}
Losses {'ner': 7378.247041811197}
Losses {'ner': 7095.02263463578}
Losses {'ner': 6929.398206653635}


##### Inspect NER predictions on one sample resume
Now that we have a trained model, let's see how it works on one of our resumes. 

In [16]:
## TODO fetch one resume out of our test dataset and store to the "resume" variable
resume = test[42]
## TODO create a spacy doc out of the resume using our trained model and save to the "doc" variable 
doc = custom_nlp(resume[0])

Now we will output the predicted entities and the existing annotated entities in that doc

In [19]:
## TODO output label and text of predicted entities (in "ents" variable of the spacy doc created above)
print("PREDICTED:")
for ent in doc.ents:
    print(ent.label_, ent)
## TODO output labeled entities (in "entities" dictionary of resume)
print("LABELED:")
for ent in resume[1]["entities"]:
    print("{} {}".format(ent[2], resume[0][ent[0]:ent[1]]))    


PREDICTED:
Name Harini Komaravelli
Location Hyderabad
Location Hyderabad
Location Hyderabad
Location Hyderabad
Location Hyderabad
Location Hyderabad
Location Hyderabad
Skills Functional Testing, Blue Prism, Qtp

ADDITIONAL INFORMATION

Area of Expertise:

➢ Familiar with Agile Methodologies.
➢ Having knowledge in Energy (Petroleum) & Health Care domains.
➢ Involved in preparation of Test Scenarios.
➢ Preparing Test Data for the test cases.

https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN
https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN


➢ Experienced in development and execution of Test cases effectively.
➢ Experienced in Functional testing, GUI testing, Smoke testing, Regression testing and
Integration Testing
➢ Experienced in doing Accessibility testing of an application
➢ Ability to understand user Requirements, Functional and Design specifications.
➢ Good knowledge of SDLC an

#### Evaluation Metrics for NER
Now that we can predict entities using our trained model, we can compare our predictions with the original annotations in our training data to evaluate how well our model performs for our task. The original annotations have been annotated manually by human annotators, and represent a "Gold Standard" against which we can compare our predictions. 

For most classification tasks, the most common evaluation metrics are:
* accuracy
* precision
* recall
* f1 score

In order to understand these metrics, we need to understand the following concepts:
* True positives - How many of the predicted entities are "true" according to the Gold Standard? (training annotation) 
* True negatives - How many entities did the model not predict which are actually not entities according to the Gold Standard?
* False positives - How many entities did the model predict which are NOT entities according to the Gold Standard?  
* False negatives - How many entities did the model "miss" - e.g. did not recognize as entities which are entities according to the Gold Standard? 

Before we go on, it is important that you understand true/false positives/negatives as well as the evaluation metrics above. Take some time now to research the web in order to find answers to the following questions:

##### Question: How are the evaluation metrics above defined in the context of evaluating Machine Learning models? How do they relate to True/False Positives/Negatives above? Please provide an intuitive description as well as the mathmatical formula for each metric. 
    * precision = true_positive / (true_positive + false_positive)
      Describes a factor of proper recognized entities to all recognized entities
    * recall = true_positive / (true_positive + false_neagtive)
      Describes a factor of proper recognized entities to all “real” entities
    * F1 = 2 * (precision * recall)/(precision + recall)
      Is a harmonic averege of precision and recall


##### Calculating Metrics based on token-level annotations or full entity-level. 
The concepts above are our first step toward understanding how to evaluate our model effectively. However, in NER, we need to take into account that we can calculate our metrics either based on all tokens (words) found in the document, or only on the entities found in the document.  

##### Token-Level evaluation. 
Token level evaluation evaluates how accurately did the model tag *each individual word/token* in the input. In order to understand this, we need to understand something called the "BILUO" Scheme (or BILOU or BIO). The spacy docs have a good reference. Please read and familiarize yourself with BILUO. 

https://spacy.io/api/annotation#biluo

Up to now, we have not been working with the BILUO scheme, but with "offsets" (for example: (112,150,"Email") - which says there is an "Email" entity between positions 112 and 150 in the text). We would like to be able to evaluate our models on a token-level using BILUO - so we need to convert our data to BILUO. Fortunately, Spacy provides a helper method to do this for us.

*Execute the code below to see how our "Gold Standard" and predictions for our example doc above look in BILUO scheme.* 
Note: some of the lines might be ommited for display purposes. 

In [23]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,
                                                   [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df(custom_nlp,test[0])
display(bilou_df)  


,Tokens,Predicted,True
0,Shibin,B-Name,B-Name
1,Raveendran,L-Name,L-Name
2,,O,O
3,TERRITORY,O,O
4,SALES,O,U-Skills
5,MANAGER,O,O
6,-,O,O
7,KKR,O,O
8,GROUP,O,O
9,OF,O,O


Based on this output, it should be very easy to calculate a token-level accuracy. We simply compare the "Predicted" to "True" columns and calculate what percentage are the same. 

In [25]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]
## accuracy is the length of this subset divided by the length of bilou_df
accuracy = same_df.shape[0] / bilou_df.shape[0]
print("Accuracy on one resume: ",accuracy)


Accuracy on one resume:  0.9851222104144527


The accuracy might seem pretty good... if it is not 100%, then let's print out those tokens where the model predicted something different than the gold standard by running the code below. 

Note - if your score on one doc is 100%, pick another document and re-run the last few cells above. 

In [27]:
## TODO find all rows in bilou_df where "Predicted" not equal to "True" column. 
diff_df = bilou_df[bilou_df["Predicted"] != bilou_df["True"]]
display(diff_df)

,Tokens,Predicted,True
4,SALES,O,U-Skills
12,Calicut,O,U-Location
60,SALES,O,U-Skills
214,SALES,O,U-Skills
545,Mumbai,U-Location,O
622,Calicut,O,U-Location
724,Calicut,O,U-Location
728,Calicut,O,U-Location
750,RETAIL,U-Skills,-
758,RETAIL,B-Skills,-


Now let's calculate the accuracy on all our test resumes and average them for an accuracy score. 

Please complete the code below to report an accuracy score on our test resumes

In [30]:
import numpy as np
doc_accuracy = []
for res in test:
    ## TODO calculate accurac for each 'res' and append to doc_accuracy list 
    res_df = make_bilou_df(custom_nlp, res)
    same_df = res_df[res_df["Predicted"] == res_df["True"]]
    accuracy = float(same_df.shape[0]) / res_df.shape[0]
    doc_accuracy.append(accuracy)

## TODO calculate mean/average of doc_accuracy (Tip: use numpy!)
total_acc = np.mean(doc_accuracy)
print("Accuracy: ",total_acc)

    

Accuracy:  0.874387119850841


##### Question: how does the model perform on token-level accuracy? What did it miss? In those cases where the predictions didn't match the gold standard, were the predictions plausible or just "spurious" (wrong)? 
*Answer here* 

##### Question: What might the advantages and disadvantages be of calculating accuracy on token-level? Hint: think about a document with 1000 tokens where only 10 tokens are annotated as entities. What might the accuracy be on such a document?  

##### Entity-Level evaluation #####
Another method of evaluating the performance of our NER model is to calculate metrics not on token-level, but on entity level. There is a good blog article that describes this method. 

http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The article goes into some detail, the most important part is the scenarios described in the section "Comparing NER system output and golden standard". 

##### Question: how do the first 3 scenarios described in the section "Comparing NER system output and golden standard" correlate to  true/false positives/negatives? 
*Answer here* 

##### Precision, Recall, F1 #####

Now we would like to calculate precision, recall, and f1 for each entity type we are interested in (our chosen entities). To do this, we need to understand the formulas for each. A good article for this is https://skymind.ai/wiki/accuracy-precision-recall-f1. 

##### Question: how can we calculate precision, recall and f1 score based on the information above? Please provide the formulas for each #####
*Answer here*

Now supply code below which calculates precision and recall and F1 on our test data for each entity type we are interested in. 



In [38]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for tres in test:
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 
        tres_df = make_bilou_df(custom_nlp, tres)
        ## calculate true false positives and false negatives for each resume
        tp = tres_df[(tres_df["Predicted"] == tres_df["True"]) & (tres_df["Predicted"].str.contains(label))]
        fp = tres_df[(tres_df["Predicted"] != tres_df["True"]) & (tres_df["Predicted"].str.contains(label))]
        fn = tres_df[(tres_df["Predicted"] != tres_df["True"]) & (tres_df["True"].str.contains(label))]
        ## aggregate result for each resume to totals
        true_positives = tp.shape[0]
        false_positives = fp.shape[0]
        false_negatives = fn.shape[0]
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out
    precision = 0.0
    recall = 0.0
    if true_positives:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives+false_negatives)

    f1 = 0.0
    if precision+recall:
        f1 = 2*((precision * recall) / (precision + recall))
    #print("Precision: ",precision)
    #print("Recall: ",recall)
    #print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame(data, index=chosen_entity_labels, columns=["Precision", "Recall", "f1"])
display(metric_df)

For label 'Skills' tp: 41 fp: 12 fn: 2
For label 'Name' tp: 2 fp: 0 fn: 0
For label 'Location' tp: 1 fp: 1 fn: 2


,Precision,Recall,f1
Skills,0.773585,0.953488,0.854167
Name,1.000000,1.000000,1.000000
Location,0.500000,0.333333,0.400000


Now we compute an average score for each metric. 

In [ ]:
## TODO compute average metrics And print them out. Use pandas dataframe "mean" method to do this
pass

##### Question: how do the average metrics here (computed on entity-level) compare to the token-level accuracy score above? Which metric(s) would you prefer to use to evaluate the quality of your model? Why? 

We are almost Done with part II! We just need to save our BILUO training data for reuse in Part III. 
For part III we are using flair and loading our data from a .csv file into a flair "Corpus". This is described here:

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md#reading-your-own-sequence-labeling-dataset

We need to create "train" and "test" .csv files using our train and test dataset which corresponds to the format described above. This format is one line containing (minimally) a text token and a NER Tag. These should be separated by whitespace. 

##### "Sentences" #####
Flair works with "Sentences" which is a list of tokens. If we simply write out our csv with one line for every token in our dataset, we will have 1 giant sentence with many thousands of words.. This is not what we want. 
We would like to partition our data so that we have a list of "Sentences" - corresponding to our intuition for a sentence - a sequence of words that belong together and is not all to long, usually separated by some punctuation. 
When we create our .csv strings/files, we need to do so so that they represent a list of sentences, each sentence consisting of a list of tokens/tags (each token/tag being one line in our csv). 

***To do this, create a blank newline after each sentence.*** 

How you split your resume data into sentences is your decision however.  

Some starter code is provided below. Complete it to create two strings which you will save as .csv. Each file will be a (long) list of token/tag lines, with sentences separated by newlines. 

In [ ]:
def bilou_for_flair(nlp, train, test):
    """
    make .csv strings from train and test for use in flair
    """    
    print("Make bilou dfs")
    ## makes a list of pandas dataframes, one for each resume. 
    training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]
    test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]
    print("Done!")
    ## strings to return
    training_file = ""
    test_file = ""
    for idx,df in enumerate(training_data_as_bilou):
        ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
        ## TODO - insert newlines after each "sentence" 
        ## You can use df.to_csv below to create a csv for your data
        ## as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        ##
        ## training_file = TODO append csv string for each resume to big training file (dont forget newline after each)
    for idx, df in enumerate(test_data_as_bilou):
        ## TODO same as above but for testfile
        pass
        
    return training_file,test_file

training,test = bilou_for_flair(custom_nlp,train,test)
## TODO save training and test as .csv (wherever you want)

Now let's load the data we persisted with flair before we go on

In [ ]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

# folder where training and test data are
data_folder = None
# your training file name
train_file = None

# your training file name
test_file = None

columns = ## TODO how is your file structured? {}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)

If you could load the corpus without error, you are ready to go on to part 3, where we will work with flair nlp!

If you have something like between 5 and 10k training sentences, and roughly a third of that as